In [92]:
import pandas as pd
import os
import glob
import chardet
import numpy as np
import math
# coding=utf-8

#查詢目錄位置
print( os.getcwd()  )

Folder_Path = "Z:/VPA-13C OAD Data/1_NG"   #要拼接的文件夾及其完整路徑，注意不要包含中文
SaveFile_Path = 'C:/Users/1907075/Project/VAS_Project_Python'    #拼接後要保存的文件路徑
SaveFile_Name = 'all_0720_NG.csv'
my_encoding = 'utf_16'

Z:\VPA-13C OAD Data\1_NG


In [94]:
#修改當前工作目錄
os.chdir(Folder_Path)
#將該文件夾下的所有文件名存入一個列表
file_list = os.listdir()

for i,j in enumerate(file_list):
    #讀取第一個CSV文件並包含表頭
    if i == 0:
        dfname = pd.read_csv(Folder_Path +'/'+ j, encoding=my_encoding, sep='\t')
    else:
        dfname = pd.read_csv(Folder_Path +'/'+ j, encoding=my_encoding, sep=',')
    filename = j
    dfname.insert(1,'filename' ,filename ) #插入檔名
    #dfname[np.isnan(dfname['Vacuum Data_VG001_Exponent'])]=0

    # Vacuum Align System (真空貼合系統)
    # 換算壓力值公式 : log10(Mantissa*10^Exponent)，總共7組 gauge pressure sensor ( Mantissa(尾數) ; Exponent(指數))
    dfname.insert(2,'VG001' ,np.log10(dfname['Vacuum Data_VG001_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG001_Exponent']) )  )  ))   #插入VG001
    dfname.insert(3,'VG002' ,np.log10(dfname['Vacuum Data_VG002_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG002_Exponent']) )  )  ))   #插入VG002
    dfname.insert(4,'VG003' ,np.log10(dfname['Vacuum Data_VG003_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG003_Exponent']) )  )  ))   #插入VG003    
    dfname.insert(5,'VG111' ,np.log10(dfname['Vacuum Data_VG111_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG111_Exponent']) )  )  ))   #插入VG111        
    dfname.insert(6,'VG921' ,np.log10(dfname['Vacuum Data_VG921_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG921_Exponent']) )  )  ))   #插入VG921        
    dfname.insert(7,'VG922' ,np.log10(dfname['Vacuum Data_VG922_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG923_Exponent']) )  )  ))   #插入VG922        
    dfname.insert(8,'VG923' ,np.log10(dfname['Vacuum Data_VG923_Mantissa']*(np.power(10.,(dfname['Vacuum Data_VG922_Exponent']) )  )  ))   #插入VG923        
    
    #取代上午/下午
    # df.loc[df.First_name != 'Bill', 'name_match'] = 'Mis-Match'  
    dfname.loc[dfname.DateTime.str.contains("上午") , 'DateTime24'] =  dfname.DateTime.str.replace('上午','') +"AM" 
    dfname.loc[dfname.DateTime.str.contains("下午") , 'DateTime24'] =  dfname.DateTime.str.replace('下午','') +"PM"        

    #轉換 DateTime 24小時制
    dfname.DateTime24 = pd.to_datetime( dfname.DateTime24 ).dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # 移動排序 DateTime24
    cols = list(dfname)
    cols.insert(0,cols.pop(cols.index('DateTime24')))
    dfname = dfname.loc[:,cols]
    
    # dfname['new_DateTime']=dfname.new_DateTime(1)

    # Row 排序
    #dfname.sort(['DateTime'], ascending=[1])
    dfname.sort_values(by=['filename', 'DateTime24'] )  
    
    #  刪除重複row    
    dfname.drop_duplicates( ['filename', 'DateTime24' ] , keep = 'first' ,inplace=True)   
    
    # 插入rownum
    dfname.insert(1,'rownum' , np.arange(len(dfname)) )
    dfClean = dfname
    dfClean.to_csv(SaveFile_Path+'/'+ SaveFile_Name,encoding="utf_8",index=False, header=True, mode='a+')  # 儲存 Column Name
   